## Problem Set 2
We are going to be using data on health care expenditures to do some plotting and fitting of a few different distributions from the Gamma family using MLE.

In [1]:
# Imports:
import pandas as pd
import numpy as np
import scipy.stats as sts
import scipy.special as spc
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [ ]:
# bring in our data
